In [5]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

### Intial Data Cleaning & Feature Engineering

In [3]:
df = pd.read_excel(r"H:\CampusX_DS\week43 - My Projects Aug 2024\end-to-end-used-car-price-prediction\cars24_final_data.xlsx")

In [4]:
df.head()

,content.appointmentId,content.make,content.model,content.variant,content.year,content.transmission,content.bodyType,content.fuelType,content.ownerNumber,content.odometerReading,...,AmbientLighting,SunroofMoonroof,SunroofType,WirelessChargingPad,VentilatedSeatsRear,HVACControl,360DegreeCamera,ParkingAssistFront,ParkingAssistSide,DriverSeatAdjustmentElectric
0,10195139789,Renault,TRIBER,RXL MT,2023,Manual,SUV,Petrol,1,6843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10572636732,Tata,Tiago,XZ PETROL,2020,Manual,Hatchback,Petrol,2,42313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10572437786,Hyundai,Xcent,SX 1.2,2017,Manual,Sedan,Petrol,1,60964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10125836738,Maruti,IGNIS,SIGMA 1.2,2022,Manual,Hatchback,Petrol,1,35102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10043035789,Tata,NEXON,XZ PLUS PETROL,2022,Manual,SUV,Petrol,1,22894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# display max row & columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)

In [7]:
#drop all duplicate rows
df.drop_duplicates(inplace=True)

In [8]:
## while scraping data from cars24, I've noticed that for some cars specs details are not available so we can avoid those
## thats why I've added a column "specs_tag" with values available & not available which can be used to filter those rows

df = df[df['specs_tag'] == "available"]

In [9]:
# It looks like there are rows where Top features are also unavailable so we can remove rows where
# all specs missing even after specs_tag filter to avoid wrong data
# aplying more filter to remove all rows where all specs are missing meaning spec features is not available on website

df = df[df[['Displacementcc', 'GearBoxNumberOfGears']].notna().all(axis=1)]

In [10]:
df.drop(columns=['TransmissionType'], inplace=True)

In [11]:
## while assesing data, I've noticed rows where data contains 1.0, 0.0 & NaN.. where 1.0 means availble,
## 0.0 measn not available & NaN means not availble, after checking I got to know its bcoz that data is not
## available on the website like even though car contains AC it says NaN bcoz data is not available on website
## which will create problem at the time of data analysis & will hamper prediction results so we are removing all those
## NaN rows where column contains all & only 3 values [1.0,0.0,NaN]


def is_valid_column(col):
    unique_values = set(col.dropna())  # Exclude NaN for checking unique values
    return unique_values.issubset({1.0, 0.0}) and \
           all(val in unique_values for val in [1.0, 0.0]) and \
           col.isnull().any()  # Ensure at least one NaN is present

# Identify columns that meet the criteria
valid_columns = df.columns[df.apply(is_valid_column, axis=0)]

# Create a new DataFrame with only the valid columns
filtered_df = df[valid_columns]

In [12]:
df.shape

(2719, 103)

In [13]:
df.drop(columns=filtered_df.columns.tolist(), inplace=True)

In [14]:
df.shape

(2719, 80)

In [15]:
## removing filter column which is of no use in analysis & model building
df.drop(columns=['specs_tag'], inplace=True)

In [16]:
from datetime import datetime

# Convert to datetime
df['content.fitnessUpto'] = pd.to_datetime(df['content.fitnessUpto'], format='%d-%b-%Y')

# Get today's date
today = pd.to_datetime(datetime.now().date())

df['content.fitnessUpto_months_remaining'] = ((df['content.fitnessUpto'].dt.year - today.year) * 12 + 
                          (df['content.fitnessUpto'].dt.month - today.month))

In [20]:
# Convert Unix timestamp to datetime
df['content.insuranceExpiry'] = pd.to_datetime(df['content.insuranceExpiry'], unit='s')

# Get today's date
today = pd.to_datetime(datetime.now().date())

# Calculate the number of months remaining
df['content.insuranceExpiry_months_remaining'] = ((df['content.insuranceExpiry'].dt.year - today.year) * 12 + (df['content.insuranceExpiry'].dt.month - today.month))

# Change all values less than 0 to 0
df['content.insuranceExpiry_months_remaining'] = df['content.insuranceExpiry_months_remaining'].clip(lower=0)

In [22]:
# Convert to datetime (auto-detect format)
df['content.lastServicedAt'] = pd.to_datetime(df['content.lastServicedAt'])

# Get today's date
today = pd.to_datetime(datetime.now().date())

# Calculate the number of months remaining since the last service
df['content.lastServicedAt_months_remaining'] = ((df['content.lastServicedAt'].dt.year - today.year) * 12 + (df['content.lastServicedAt'].dt.month - today.month))

# Change all negative values to positive (use abs())
df['content.lastServicedAt_months_remaining'] = df['content.lastServicedAt_months_remaining'].abs()

In [23]:
## removing columns as already been feature engineered

df.drop(columns=['content.fitnessUpto', 'content.insuranceExpiry','content.lastServicedAt'], inplace=True)

In [ ]:
# remove content.city
df.drop(columns=['content.city'], inplace=True)

In [26]:
# extract state code from content.cityRto
df['content.staterto'] = df['content.cityRto'].str[:2]

In [27]:
df.drop(columns=['content.staterto'], inplace=True)

In [28]:
#change NaN to "not available" for selected columns
cols_to_replace = ['ABSAntilockBrakingSystem', 'PowerWindowsFront', 'PowerWindowsRear', 
                   'AirConditioner', '12VPowerOutlet', 'MultifunctionDisplayScreen', 
                   'EntertainmentDisplayScreen', 'VoiceRecognition', 'SmartCardSmartKey', 
                   'AmbientLighting', 'SunroofMoonroof', 'WirelessChargingPad', 
                   'VentilatedSeatsRear', 'HVACControl', '360DegreeCamera', 
                   'ParkingAssistSide', 'DriverSeatAdjustmentElectric']

df[cols_to_replace] = df[cols_to_replace].fillna("not available")

In [29]:
#missing values in HeadlampBulbTypeHighBeam can be filled using HeadlampBulbTypeLowBeam column
df['HeadlampBulbTypeHighBeam'] = df['HeadlampBulbTypeHighBeam'].fillna(df['HeadlampBulbTypeLowBeam'])

In [30]:
#strip whole dataset
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [31]:
#Change all not available to 0 for better visualization
# To ensure that the changes are applied only when the columns contain exactly two unique values as specified
def update_columns(df):
    for col in df.columns:
        unique_values = set(df[col].astype(str).unique())
#         print(f"Processing column: {col}, Unique values: {unique_values}")  # Debugging line
        
        if unique_values == {'1', 'not available'}:
            df[col] = df[col].replace('not available', 0)
        
        elif unique_values == {'Available', 'Not available'}:
            df[col] = df[col].replace({'Available': 1, 'Not available': 0})
            
        elif unique_values == {'0', 'not available'}:
            df[col] = df[col].replace('not available', 0)

In [32]:
# Apply the function to your DataFrame
update_columns(df)

In [33]:
# Change content.duplicatekey True/False to 1/0
df['content.duplicateKey'] = df['content.duplicateKey'].replace({True: 1, False: 0})

In [34]:
# Extract State from cityrto
df['car_state'] = df['content.cityRto'].str.strip().str[:2]

In [35]:
# content.cityRto content.registrationNumber content.listingPrice - remove
df.drop(columns=['content.cityRto','content.registrationNumber','content.listingPrice'], inplace=True)

In [36]:
df.shape

(2719, 76)

In [37]:
df.sample(5)

,content.appointmentId,content.make,content.model,content.variant,content.year,content.transmission,content.bodyType,content.fuelType,content.ownerNumber,content.odometerReading,content.onRoadPrice,content.insuranceType,content.duplicateKey,Airbags,ABSAntilockBrakingSystem,InstrumentPanelType,NumberOfSpeakers,AirConditioner,PowerWindowsFront,PowerWindowsRear,SteeringWheelMaterial,ParkingAssistRear,12VPowerOutlet,SeatUpholstery,HeadlampLensType,HeadlampBulbTypeHighBeam,HeadlampBulbTypeLowBeam,RimTypeFrontWheels,RimTypeRearWheels,RemoteTrunkRelease,Displacementcc,Cylinders,GearBoxNumberOfGears,NumberOfDiscBrakes,GroundClearancemm,SeatingCapacity,Bootspacelitres,Widthmm,Lengthmm,WheelBasemm,FueltankCapacitylitres,MaxPowerbhp,MaxPowerrpm,EmissionStandard,MaxTorqueNm,defects,repainted,perfectParts,Left Front Tyre,Right Front Tyre,Left Rear Tyre,Right Rear Tyre,Spare Tyre,MultifunctionDisplayScreen,MultifunctionDisplayScreenSizein,EntertainmentDisplayScreen,EntertainmentDisplayScreenSizein,NCAPRating,VoiceRecognition,SpeakerBrand,SmartCardSmartKey,GPSNavigationSystem,AmbientLighting,SunroofMoonroof,SunroofType,WirelessChargingPad,VentilatedSeatsRear,HVACControl,360DegreeCamera,ParkingAssistFront,ParkingAssistSide,DriverSeatAdjustmentElectric,content.fitnessUpto_months_remaining,content.insuranceExpiry_months_remaining,content.lastServicedAt_months_remaining,car_state
4994,14448580761,Hyundai,Grand i10,SPORTZ 1.2 KAPPA VTVT,2018,Manual,Hatchback,Petrol,2,61847,707525,3rd Party,1,2.0,1.0,Analogue & Digital,4.0,1.0,1.0,1.0,Plastic,Sensor & Camera,1.0,Vinyl,Complex Surface,Halogen,Halogen,Steel,Steel,1.0,1197.0,4.0,5.0,2.0,165.0,5.0,NaN,1660.0,3765.0,2425.0,43.0,83.0,6000.0,NaN,113.0,7,7.0,NaN,OK,OK,OK,OK,OK,1.0,7.0,1.0,7.0,NaN,1.0,NaN,not available,NaN,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,105,2,2,BR
3067,13513635740,Hyundai,NEW SANTRO,MAGNA,2020,Manual,Hatchback,Petrol,2,18415,595182,3rd Party,0,1.0,1.0,Analogue & Digital,2.0,1.0,1.0,1.0,Plastic,Sensor,1.0,Cloth,Complex Surface,Halogen,Halogen,Steel,Steel,1.0,1086.0,4.0,5.0,2.0,170.0,5.0,235.0,1645.0,3610.0,2400.0,35.0,69.0,5500.0,BSVI,99.0,8,1.0,NaN,WARN,OK,OK,OK,OK,not available,NaN,not available,NaN,NaN,not available,NaN,not available,NaN,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,128,0,3,GJ
4348,10245330716,Renault,Kwid,CLIMBER 1.0 AMT,2017,Automatic,Hatchback,Petrol,2,89552,542210,Comprehensive,0,1.0,not available,Full Digital,2.0,1.0,1.0,not available,Leather Covered,NaN,1.0,Upgraded Cloth/Velour,Complex Surface,Halogen,Halogen,Steel,Steel,1.0,999.0,3.0,5.0,2.0,180.0,5.0,300.0,1579.0,3679.0,2422.0,28.0,68.0,5500.0,NaN,91.0,9,0.0,NaN,WARN,WARN,WARN,WARN,OK,not available,NaN,not available,NaN,NaN,not available,NaN,not available,3D And Voice,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,92,6,4,KA
691,10066335751,Tata,Tiago,XT PETROL,2023,Manual,Hatchback,Petrol,1,6933,699448,Comprehensive,1,2.0,1.0,Full Digital,4.0,1.0,1.0,1.0,Plastic,Sensor,1.0,Upgraded Cloth/Velour,Complex Surface,Halogen,Halogen,Steel,Steel,1.0,1199.0,3.0,5.0,2.0,170.0,5.0,242.0,1677.0,3765.0,2400.0,35.0,86.0,6000.0,BSVI,113.0,4,6.0,NaN,OK,OK,OK,OK,OK,not available,NaN,not available,NaN,4.0,not available,NaN,not available,NaN,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,160,4,3,DL
3994,10400336732,Tata,NEXON,XZA PLUS (O) PETROL DARK EDITION,2022,Automatic,SUV,Petrol,1,56295,1494378,3rd Party,0,2.0,1.0,Full Digital,8.0,1.0,1.0,1.0,Leather Covered,Sensor & Camera,1.0,Synthetic Leather,Projector Beam,Halogen,Halogen,Alloy,Alloy,1.0,1199.0,3.0,6.0,2.0,209.0,5.0,350.0,1811.0,3993.0,2498.0,44.0,120.0,5500.0,BSVI,170.0,5,1.0,NaN,OK,OK,OK,OK,OK,1.0,7.0,1.0,7.0,NaN,1.0,Harman/Kardon,1.0,NaN,not available,1.0,Electric,n

In [38]:
df.drop(columns=['content.appointmentId'], inplace=True)

In [39]:
df.sample(5)

,content.make,content.model,content.variant,content.year,content.transmission,content.bodyType,content.fuelType,content.ownerNumber,content.odometerReading,content.onRoadPrice,content.insuranceType,content.duplicateKey,Airbags,ABSAntilockBrakingSystem,InstrumentPanelType,NumberOfSpeakers,AirConditioner,PowerWindowsFront,PowerWindowsRear,SteeringWheelMaterial,ParkingAssistRear,12VPowerOutlet,SeatUpholstery,HeadlampLensType,HeadlampBulbTypeHighBeam,HeadlampBulbTypeLowBeam,RimTypeFrontWheels,RimTypeRearWheels,RemoteTrunkRelease,Displacementcc,Cylinders,GearBoxNumberOfGears,NumberOfDiscBrakes,GroundClearancemm,SeatingCapacity,Bootspacelitres,Widthmm,Lengthmm,WheelBasemm,FueltankCapacitylitres,MaxPowerbhp,MaxPowerrpm,EmissionStandard,MaxTorqueNm,defects,repainted,perfectParts,Left Front Tyre,Right Front Tyre,Left Rear Tyre,Right Rear Tyre,Spare Tyre,MultifunctionDisplayScreen,MultifunctionDisplayScreenSizein,EntertainmentDisplayScreen,EntertainmentDisplayScreenSizein,NCAPRating,VoiceRecognition,SpeakerBrand,SmartCardSmartKey,GPSNavigationSystem,AmbientLighting,SunroofMoonroof,SunroofType,WirelessChargingPad,VentilatedSeatsRear,HVACControl,360DegreeCamera,ParkingAssistFront,ParkingAssistSide,DriverSeatAdjustmentElectric,content.fitnessUpto_months_remaining,content.insuranceExpiry_months_remaining,content.lastServicedAt_months_remaining,car_state
3969,Maruti,Dzire,VDI AMT,2017,Automatic,Sedan,Diesel,1,95453,957500,Comprehensive,0,2.0,1.0,Analogue & TFT,4.0,1.0,1.0,1.0,Plastic,NaN,1.0,Upgraded Cloth/Velour,Complex Surface,Halogen,Halogen,Steel,Steel,1.0,1248.0,4.0,5.0,2.0,163.0,5.0,378.0,1735.0,3995.0,2450.0,37.0,75.0,4000.0,NaN,190.0,2,3.0,NaN,WARN,WARN,WARN,WARN,WARN,not available,NaN,not available,NaN,NaN,not available,NaN,not available,NaN,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,96,0,5,TS
100,Maruti,Swift,LXI,2018,Manual,Hatchback,Petrol,1,53580,603900,Comprehensive,0,2.0,1.0,Analogue & Digital,NaN,1.0,not available,not available,Plastic,NaN,1.0,Cloth,Complex Surface,Halogen,Halogen,Steel,Steel,1.0,1197.0,4.0,5.0,2.0,163.0,5.0,268.0,1735.0,3840.0,2450.0,37.0,83.0,6000.0,NaN,113.0,23,7.0,NaN,OK,OK,WARN,WARN,WARN,not available,NaN,not available,NaN,NaN,not available,NaN,not available,NaN,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,109,0,3,HR
3716,Mahindra,XUV500,W11 (O) AT,2019,Automatic,SUV,Diesel,2,63817,2408587,Zero Depreciation,1,6.0,1.0,Analogue & Digital,6.0,1.0,1.0,1.0,Alloy & Leather,Sensor & Camera,1.0,Leather,Projector Beam,Halogen,Halogen,Alloy,Alloy,1.0,2179.0,4.0,6.0,4.0,160.0,7.0,93.0,1890.0,4585.0,2700.0,70.0,155.0,3750.0,NaN,360.0,0,1.0,NaN,WARN,OK,OK,WARN,WARN,1.0,7.0,1.0,7.0,NaN,1.0,NaN,1.0,3D And Voice,not available,1.0,Electric,not available,not available,not available,not available,NaN,not available,1.0,115,6,2,MH
4620,Hyundai,Verna,FLUIDIC 4S 1.6 VTVT S,2016,Manual,Sedan,Petrol,1,75233,1008405,Comprehensive,0,1.0,1.0,NaN,6.0,1.0,1.0,1.0,Plastic,Sensor & Camera,not available,Cloth,Complex Surface,Halogen,Halogen,Alloy,Alloy,NaN,1591.0,4.0,5.0,2.0,165.0,5.0,NaN,1700.0,4375.0,2570.0,43.0,123.0,6300.0,NaN,154.0,28,7.0,NaN,WARN,OK,OK,OK,OK,not available,NaN,not available,NaN,NaN,not available,NaN,not available,NaN,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,76,4,2,TN
3974,Maruti,Swift,ZXI,2017,Manual,Hatchback,Petrol,1,49184,759211,3rd Party,0,2.0,1.0,Analogue & Digital,6.0,1.0,1.0,1.0,Plastic,Sensor,1.0,Cloth,Complex Surface,Halogen,Halogen,Alloy,Alloy,1.0,1197.0,4.0,5.0,2.0,170.0,5.0,204.0,1695.0,3850.0,2430.0,42.0,84.0,6000.0,NaN,115.0,0,5.0,NaN,OK,OK,OK,OK,OK,not available,NaN,not available,NaN,NaN,not available,NaN,1.0,NaN,not available,not available,NaN,not available,not available,not available,not available,NaN,not available,not available,90,4,2,TS


In [40]:
# Replace "OK" with 1 and "WARN" with 0 in specified columns
columns_to_replace = ['Left Front Tyre', 'Right Front Tyre', 'Left Rear Tyre', 'Right Rear Tyre', 'Spare Tyre']
df[columns_to_replace] = df[columns_to_replace].replace({'OK': 1, 'WARN': 0})

In [41]:
# Calculate the sum of 'OK' values
df['Tyre_Health'] = df[['Left Front Tyre', 'Right Front Tyre', 'Left Rear Tyre', 'Right Rear Tyre', 'Spare Tyre']].sum(axis=1)

# Calculate the total number of tyres (columns)
total_tyres = df[['Left Front Tyre', 'Right Front Tyre', 'Left Rear Tyre', 'Right Rear Tyre', 'Spare Tyre']].shape[1]

# Calculate the percentage of 'OK' values
df['tyre_health_pct'] = (df['Tyre_Health'] / total_tyres) * 100

In [42]:
df.drop(columns=['Left Front Tyre', 'Right Front Tyre', 'Left Rear Tyre', 'Right Rear Tyre', 'Spare Tyre', 'Tyre_Health'], inplace=True)

In [43]:
df = df[['ABSAntilockBrakingSystem','AirConditioner','Airbags','Bootspacelitres','Displacementcc','FueltankCapacitylitres','GroundClearancemm','tyre_health_pct','MaxPowerbhp','MaxPowerrpm','MaxTorqueNm','SeatingCapacity','content.bodyType','content.duplicateKey','content.fitnessUpto_months_remaining','content.fuelType','content.insuranceExpiry_months_remaining','content.insuranceType','content.make','content.odometerReading','content.ownerNumber','content.transmission','content.year','defects','repainted','content.onRoadPrice']]

In [44]:
df.sample(5)

,ABSAntilockBrakingSystem,AirConditioner,Airbags,Bootspacelitres,Displacementcc,FueltankCapacitylitres,GroundClearancemm,tyre_health_pct,MaxPowerbhp,MaxPowerrpm,MaxTorqueNm,SeatingCapacity,content.bodyType,content.duplicateKey,content.fitnessUpto_months_remaining,content.fuelType,content.insuranceExpiry_months_remaining,content.insuranceType,content.make,content.odometerReading,content.ownerNumber,content.transmission,content.year,defects,repainted,content.onRoadPrice
3504,1.0,1.0,2.0,268.0,1197.0,37.0,163.0,100.0,90.0,6000.0,113.0,5.0,Hatchback,0,165,Petrol,8,Comprehensive,Maruti,3451,1,Automatic,2023,0,0.0,1044300
4395,1.0,1.0,6.0,311.0,1197.0,37.0,NaN,80.0,83.0,6000.0,115.0,5.0,Hatchback,0,141,Petrol,0,3rd Party,Hyundai,31179,1,Manual,2021,2,3.0,1153705
3408,1.0,1.0,2.0,345.0,1199.0,37.0,165.0,100.0,86.0,6000.0,113.0,5.0,Hatchback,0,127,Petrol,0,3rd Party,Tata,23825,1,Manual,2020,8,0.0,967482
4622,1.0,1.0,1.0,NaN,1197.0,43.0,165.0,60.0,83.0,6000.0,113.0,5.0,Hatchback,0,99,Petrol,3,Zero Depreciation,Hyundai,15517,1,Automatic,2018,6,1.0,750176
3044,not available,1.0,0.0,NaN,1197.0,43.0,NaN,0.0,83.0,6000.0,113.0,5.0,Hatchback,0,58,Petrol,0,Comprehensive,Hyundai,47742,1,Manual,2014,21,3.0,625679


In [45]:
df.isnull().sum()

ABSAntilockBrakingSystem                      0
AirConditioner                                0
Airbags                                     286
Bootspacelitres                             491
Displacementcc                                0
FueltankCapacitylitres                        6
GroundClearancemm                           530
tyre_health_pct                               0
MaxPowerbhp                                   0
MaxPowerrpm                                   0
MaxTorqueNm                                   0
SeatingCapacity                               1
content.bodyType                              0
content.duplicateKey                          0
content.fitnessUpto_months_remaining          0
content.fuelType                              0
content.insuranceExpiry_months_remaining      0
content.insuranceType                         0
content.make                                  0
content.odometerReading                       0
content.ownerNumber                     

In [46]:
## incorporate above cleaning & feature engineering in data_ingestion.py

In [54]:
df['ABSAntilockBrakingSystem'] = pd.to_numeric(df['ABSAntilockBrakingSystem'].str.replace("not available", "0"), errors='coerce')

### Pre-processing

In [86]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from src.logger.logging import logging
from src.exception.exception import customexception
import os
import sys
from dataclasses import dataclass
from src.utils.utils import save_object

def select_best_k(X: pd.DataFrame, y: pd.Series):
    """
    Select the best k for KNNImputer using cross-validation, using only numerical columns since cat is encoded.
    """
    best_k = None
    best_score = -1

    # Test k values from 1 to 10
    for k in range(1, 11):
        try:
            # Initialize the KNN imputer
            knn_imputer = KNNImputer(n_neighbors=k)

            # Only use numerical columns for KNNImputer
            X_imputed = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

            # Evaluate model using cross-validation
            pipeline = Pipeline(steps=[
                    ('scaler', StandardScaler()),
                    ('regressor', RandomForestRegressor(random_state=42))
                ])

            cv = KFold(n_splits=5, shuffle=True, random_state=42)
            score = cross_val_score(pipeline, X_imputed, y, cv=cv, scoring='r2').mean()

            logging.info(f'k: {k}, Cross-validated score: {score:.4f}')
                
            if score > best_score:
                best_score = score
                best_k = k

        except Exception as e:
            logging.error(f"Error in selecting best k: {k}")
            raise customexception(e, sys)

        logging.info(f'Best k: {best_k} with score: {best_score:.4f}')
        return best_k

In [87]:
cat_cols = ['content.bodyType','content.fuelType','content.insuranceType','content.make','content.transmission','content.duplicateKey']
num_cols = df.columns.difference(cat_cols).tolist()

In [88]:
best_k = select_best_k(X = df[num_cols], y = df['content.onRoadPrice'])

In [94]:
def get_data_preprocessing(X: pd.DataFrame, y: pd.Series):
    try:
        logging.info('Preprocessing initiated')

        # Define categorical and numerical columns
        cat_cols = ['content.bodyType', 'content.fuelType', 'content.insuranceType', 'content.make', 'content.transmission', 'content.duplicateKey']
        num_cols = X.columns.difference(cat_cols).tolist()

        logging.info(f"Categorical Columns: {cat_cols}")
        logging.info(f"Numerical Columns: {num_cols}")

        # Find the optimal n_neighbors for KNNImputer using cross-validation (pass only X[num_cols] for KNN)
        best_k = select_best_k(X[num_cols], y)  # Pass y for target column separately

        # Numerical Pipeline using KNNImputer with best k
        num_pipeline = Pipeline(
                steps=[("imputer", KNNImputer(n_neighbors=best_k)),  # KNN imputation
                    ("scaler", StandardScaler())]  # Standard Scaling
            )

        # Categorical Pipeline using OneHotEncoder followed by KNNImputer
        cat_pipeline = Pipeline(
                steps=[("encoder", OneHotEncoder(sparse_output=False, drop='first')),  # OneHotEncoding
                    ("imputer", KNNImputer(n_neighbors=best_k))]  # KNN imputation
            )

        # Create the preprocessor using ColumnTransformer
        preprocessor = ColumnTransformer(
                transformers=[('num_pipeline', num_pipeline, num_cols),
                            ('cat_pipeline', cat_pipeline, cat_cols)]
            )
            
        return preprocessor

    except Exception as e:
        logging.error("Exception occurred in get_data_preprocessing")
        raise customexception(e, sys)

In [95]:
train_data_path = r"H:\CampusX_DS\week43 - My Projects Aug 2024\end-to-end-used-car-price-prediction\artifacts\train.xlsx"
test_data_path = r"H:\CampusX_DS\week43 - My Projects Aug 2024\end-to-end-used-car-price-prediction\artifacts\test.xlsx"

train_df = pd.read_excel(train_data_path)
test_df = pd.read_excel(test_data_path)

In [96]:
target_column_name = 'content.onRoadPrice'

In [97]:
preprocessing_obj = get_data_preprocessing(X_train, y_train)

In [98]:
drop_columns = [target_column_name]
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

In [99]:
X_train = train_df.drop(columns=[target_column_name])
y_train = train_df[target_column_name]

X_test = test_df.drop(columns=[target_column_name])
y_test = test_df[target_column_name]

In [101]:
# X_train_transformed = preprocessing_obj.fit_transform(X_train)
# X_test_transformed = preprocessing_obj.transform(X_test)

X_train_transformed = preprocessing_obj.fit_transform(X_train)  # Apply transformation to feature columns only
X_test_transformed = preprocessing_obj.transform(X_test)  # Apply transformation to feature columns only

In [102]:
train_arr = np.c_[X_train_transformed, np.array(y_train)]
test_arr = np.c_[X_test_transformed, np.array(y_test)]